**IMPORT LIBRARY**

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from google.colab import files
import os
from tensorflow.keras.preprocessing import image

# Verifikasi penggunaan GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


**UPLOAD DATASET**

In [5]:
# Unggah file ZIP yang berisi dataset
uploaded = files.upload()

# Ekstrak file ZIP
import zipfile
import io

for file_name in uploaded.keys():
    with zipfile.ZipFile(io.BytesIO(uploaded[file_name]), 'r') as zip_ref:
        zip_ref.extractall('/content/dataset')


Saving dataset.zip to dataset (1).zip


**PREPROCESSING DATA**

In [6]:
# Membuat ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    '/content/dataset',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/content/dataset',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 180 images belonging to 3 classes.
Found 45 images belonging to 3 classes.


**MELATIH MODEL DENGAN METODE CONVOLUTIONAL NEURAL NETWORK**

In [7]:
# Membuat model CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Melatih model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


Epoch 1/10
6/6 [==============================] - 9s 1s/step - loss: 1.5598 - accuracy: 0.3611 - val_loss: 1.1012 - val_accuracy: 0.3333
Epoch 2/10
6/6 [==============================] - 7s 1s/step - loss: 1.1070 - accuracy: 0.3333 - val_loss: 1.0944 - val_accuracy: 0.3333
Epoch 3/10
6/6 [==============================] - 9s 1s/step - loss: 1.0884 - accuracy: 0.3389 - val_loss: 1.0879 - val_accuracy: 0.3333
Epoch 4/10
6/6 [==============================] - 8s 1s/step - loss: 1.0716 - accuracy: 0.3389 - val_loss: 1.0760 - val_accuracy: 0.3333
Epoch 5/10
6/6 [==============================] - 7s 1s/step - loss: 1.0351 - accuracy: 0.4000 - val_loss: 1.0693 - val_accuracy: 0.3333
Epoch 6/10
6/6 [==============================] - 9s 2s/step - loss: 1.0257 - accuracy: 0.4333 - val_loss: 1.0700 - val_accuracy: 0.3333
Epoch 7/10
6/6 [==============================] - 8s 1s/step - loss: 0.9197 - accuracy: 0.4889 - val_loss: 1.0085 - val_accuracy: 0.4444
Epoch 8/10
6/6 [=========================

**MENYIMPAN MODEL**

In [8]:
model.save('/content/rice_classifier_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


**PREDIKSI GAMBAR**

In [15]:
# Fungsi untuk memprediksi jenis beras dari gambar
def predict_rice(image_path, model):
    img = image.load_img(image_path, target_size=(128, 128))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    pred = model.predict(img_array)
    classes = ['Basmathi', 'IR64', 'Ketan']
    pred_percentage = { classes[i]: pred[0][i] for i in range(3) }

    return pred_percentage

# Unggah gambar baru untuk prediksi
uploaded = files.upload()

for fn in uploaded.keys():
    path = '/content/' + fn
    img = image.load_img(path, target_size=(128, 128))
    pred_percentage = predict_rice(path, model)
    print(f"Prediksi untuk {fn}:")
    for key, value in pred_percentage.items():
        print(f"{key}: {value * 100:.2f}%")


Saving B_nf082.jpg to B_nf082.jpg
1/1 [==============================] - 0s 44ms/step
Prediksi untuk B_nf082.jpg:
Basmathi: 50.50%
IR64: 41.19%
Ketan: 8.31%
